In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import taurex
import os
from taurex.log import disableLogging
disableLogging()
from taurex.cache import OpacityCache,CIACache
from taurex.data.profiles.temperature import Isothermal,Guillot2010, NPoint
from taurex.data import Planet
from taurex.data.stellar import PhoenixStar
from taurex.data.stellar import BlackbodyStar
from taurex.data.profiles.chemistry import TaurexChemistry
from taurex.data.profiles.chemistry import ConstantGas
from taurex.data.profiles.chemistry import TwoLayerGas
#from taurex.data.profiles.chemistry import ACEChemistry
from taurex.model import TransmissionModel, EmissionModel
from taurex.contributions import *
from taurex.data.spectrum.observed import ObservedSpectrum
from astropy.io.fits.verify import VerifyWarning
import warnings
warnings.simplefilter('ignore', category=VerifyWarning)
import gzip
from taurex.binning.fluxbinner import FluxBinner
from taurex.util.util import wnwidth_to_wlwidth
from scipy.interpolate import interp1d
import gzip

def create_dic(chosen_target):
        
    #print("Loading Target Properties...")
    exoplanet_database = pd.read_csv('PotentialPlanets.csv')

    exoplanet_names = pd.Index(exoplanet_database['Planet Name'])

    planet_index = exoplanet_names.get_loc(chosen_target)

    target_data = {             'star_name':                exoplanet_database['Star Name'][planet_index], \
                                'star_mass':                float(exoplanet_database['Star Mass [Ms]'][planet_index]), \
                                'star_temperature':         float(exoplanet_database['Star Temperature [K]'][planet_index]), \
                                'star_radius':              float(exoplanet_database['Star Radius [Rs]'][planet_index]), \
                                'star_logg':                float(exoplanet_database['Star log(g)'][planet_index]), \
                                'star_k_mag':               float(exoplanet_database['Star K Mag'][planet_index]), \
                                'star_metallicity':         float(exoplanet_database['Star Metallicity'][planet_index]), \
                                'star_distance':            float(exoplanet_database['Star Distance [pc]'][planet_index]), \
                                'planet_name':              exoplanet_database['Planet Name'][planet_index], \
                                'planet_period':            float(exoplanet_database['Planet Period [days]'][planet_index]), \
                                'planet_temperature':       float(exoplanet_database['Planet Temperature [K]'][planet_index]), \
                                'planet_sma':               float(exoplanet_database['Planet Semi-major Axis [m]'][planet_index]), \
                                'planet_radius':            float(exoplanet_database['Planet Radius [Re]'][planet_index]), \
                                'planet_radius_jup':            float(exoplanet_database['Planet Radius [Rj]'][planet_index]), \
                                'planet_albedo':            float(exoplanet_database['Planet Albedo'][planet_index]), \
                                'planet_mass':              float(exoplanet_database['Planet Mass [Me]'][planet_index]), \
                                'planet_mass_jup':              float(exoplanet_database['Planet Mass [Mj]'][planet_index]), \
                                'molecular_weight':         float(exoplanet_database['Molecular Weight'][planet_index]), \
                                'transit_duration':         float(exoplanet_database['Transit Duration [s]'][planet_index]), \
                                'impact_parameter':         float(exoplanet_database['Impact Parameter'][planet_index]), \
                                'eccentricity':             float(exoplanet_database['Eccentricity'][planet_index]), \
                                'periastron':               float(0), \
                                'inclination':              float(exoplanet_database['Inclination'][planet_index]), \
                                'star_ra':                  float(exoplanet_database['Star RA'][planet_index]), \
                                'star_dec':                 float(exoplanet_database['Star Dec'][planet_index]) \
                             }
    
    return target_data

def create_spec(args):
    
    OpacityCache().set_opacity_path("/Users/billye/Documents/Datasets/Linelists/xsec/TauRex_sampled_xsecs_R10000_0.3-15")
    CIACache().set_cia_path("/Users/billye/Documents/Datasets/Linelists/cia/hitran")
 
    isothermal = Isothermal(T= args['target_data']['planet_temperature'])

    planet = Planet(planet_radius = 0.925*args['target_data']['planet_radius_jup'], planet_mass = args['target_data']['planet_mass_jup'])

    star = BlackbodyStar(temperature = args['target_data']['star_temperature'],\
                       radius = args['target_data']['star_radius'],\
                       mass = args['target_data']['star_mass']
                       )

    chemistry = TaurexChemistry(fill_gases = ['H2', 'He'], ratio = 0.17)
    
    chemistry.addGas(TwoLayerGas('H', mix_ratio_surface = 1e-2, \
                                 mix_ratio_top = 0.5, mix_ratio_P = 1e4))
    
    for ii in range(len(args['molecules'])):
        chemistry.addGas(ConstantGas(args['molecules'][ii], mix_ratio=args['abundances'][ii]))
    
    
    tm = TransmissionModel(planet=planet,temperature_profile=isothermal,\
                           chemistry=chemistry,star=star, atm_min_pressure=1e-4,\
                           atm_max_pressure=1e6,nlayers=100)
        
    if args['clouds'] == 'Simple':
        tm.add_contribution(SimpleCloudsContribution(clouds_pressure = args['pressure']))
            
    for ii in range(len(args['molecules'])):   
        if args['molecules'][ii] == 'e-':
            tm.add_contribution(HydrogenIon())
    
    tm.add_contribution(AbsorptionContribution())
    tm.add_contribution(RayleighContribution())
    tm.add_contribution(CIAContribution(cia_pairs=['H2-H2', 'H2-He']))
    
    tm.build()

    tm_grid, tm_result, tm_tau, tm_extra = tm.model()
    
    tm_wl_grid = 10000./tm_grid

    tm_spectrum = pd.DataFrame(np.column_stack((tm_wl_grid, tm_result)))
    tm_spectrum.columns = ['Wavelength', 'Depth']
    
    return tm_spectrum

def bin_spectrum(low_res, high_res):
        
    transmission_spectra = high_res.sort_values(by = 'Wavelength')
    low_res = low_res.sort_values(by = 'Wavelength')
    low_res = low_res.reset_index(drop = True)
    
    #binner  = FluxBinner(low_res['Wavelength'].values, low_res['Bandwidth'].values)
    binner  = FluxBinner(low_res['Wavelength'].values)

    _, binned_transit_depth, _,  _ = binner.bindown(transmission_spectra['Wavelength'].values, transmission_spectra['Depth'].values)
    
    low_res['Depth'] = np.array(binned_transit_depth)
    
    return low_res

In [4]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random
import glob

def series(dots, colr):
    a,b=[],[]
    for i in range(dots):
        a.append(random.randint(1,100))
        b.append(random.randint(1,100))
    plt.scatter(a,b, c=colr)
    return()

def load_ariel(planet,hr):

    lr = pd.DataFrame(np.arange(0.5,8.,0.002), columns = ['Wavelength'])
    lr = bin_spectrum(lr,hr)
    
    files = np.sort(glob.glob(f'Inputs/*_{planet}.csv'))
    print(files[0])
    ariel = pd.read_csv(files[0], skiprows = 71)
    
    ariel_spec = ariel[['Wavelength','ECSignal', 'NoiseOnTransitFloor', 'BandWidth']]
    ariel_spec.columns = ['Wavelength', 'Depth', 'Error', 'Bandwidth']

    ariel_spec = bin_spectrum(ariel_spec,hr)
    
    return ariel_spec, lr
    

def fm(H2O, CH4, NH3, VO, TiO, HCN, CO, CO2, K, Na, Hminus, FeH, Clouds, Planet, Observations):
    
    args = {}
    args['target_data'] = create_dic(Planet)

    constants = {'au':1.49597e11, 'earth_radius': 6.371e6, 
                          'earth_mass': 5.972e24, 'solar_radius': 6.95508e8, 
                          'solar_mass': 1.989e30, 'pc': 3.0857e16, 
                          'h': 6.62607004E-34, 'c': 2.99792458e8, 
                          'b': 1.4387752e4,  'B': 1.191042e8, 
                          'G': 6.67408e-11, 'jupiter_radius': 69.911e6, 
                          'jupiter_mass': 1.898e27, 'jupiter_density': 638.56,
                          'k': 1.38064852e-23, 'Na': 6.02214e23}
    
    args['molecules'] = ['H2O', 'CH4', 'NH3', 'CO', 'CO2', \
                         'HCN', 'Na', 'K', 'TiO', 'VO', 'FeH', 'e-']
    
    args['abundances'] = [10**H2O, 10**CH4, 10**NH3, 10**CO, 10**CO2, \
                          10**HCN, 10**Na, 10**K, 10**TiO, 10**VO, 10**FeH, 10**Hminus]
     
    args['clouds'] = 'Simple'
    args['pressure'] = 10**Clouds
    fm = create_spec(args)
    
    ariel,lr = load_ariel(Planet,fm)
    ariel['Error'] = ariel['Error']/np.sqrt(Observations)
    
    plt.figure(figsize=(10,6))
    plt.plot(lr['Wavelength'], lr['Depth'], c = '#228B22')
    plt.errorbar(ariel['Wavelength'], ariel['Depth'], yerr = ariel["Error"], fmt = 'o', c = 'k', zorder = 10)
    plt.xlim([0.5,8])
    plt.xlabel('Wavelength [$\mu$m]')
    plt.ylabel('Transit Depth')
    #plt.xscale('log')
    
h2o = widgets.FloatSlider(value=-3, min=-15, max=-1, step=1, description='$H_2O$', continuous_update=False)
co = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$CO$', continuous_update=False)
co2 = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$CO_2$', continuous_update=False)
nh3 = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$NH_3$', continuous_update=False)
na = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$Na$', continuous_update=False)
k = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$K$', continuous_update=False)
hcn = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$HCN$', continuous_update=False)
ch4 = widgets.FloatSlider(value=-7, min=-15, max=-1, step=1, description='$CH_4$', continuous_update=False)
tio = widgets.FloatSlider(value=-15, min=-15, max=-1, step=1, description='$VO$', continuous_update=False)
vo = widgets.FloatSlider(value=-15, min=-15, max=-1, step=1, description='$TiO$', continuous_update=False)
feh = widgets.FloatSlider(value=-15, min=-15, max=-1, step=1, description='$FeH$', continuous_update=False)
hminus = widgets.FloatSlider(value=-15, min=-15, max=-1, step=1, description='$H-$', continuous_update=False)

clouds = widgets.FloatSlider(value=3, min=-4, max=6, step=1, description='$Clouds$', continuous_update=False)
obs = widgets.IntSlider(value=1, min=1, max=30, step=1, description='$Transits$', continuous_update=False)
planet = widgets.Dropdown(options=['WASP-162 b', 'Kepler-419 b', 'CoRoT-10 b', 'HAT-P-34 b',\
                                  'Kepler-1656 b', 'K2-287 b', 'WASP-117 b'], value='WASP-117 b', \
                        description='$Planet$',disabled=False)

box1 = widgets.HBox([h2o,co,co2,ch4])
box2 = widgets.HBox([nh3,hcn,na,k])
box3 = widgets.HBox([tio,vo,feh, hminus])
box4 = widgets.HBox([clouds, obs, planet])

ui = widgets.VBox([box1,box2,box3,box4])

#out = interact_manual(fm, H2O = h2o, CH4 = ch4, NH3 = nh3, Na = na, K = k, CO = co, CO2 = co2, \
#                Clouds = clouds, Planet = planet, Observations = obs)
                
                
                
out = widgets.interactive_output(fm, {'H2O': h2o, 'CH4': ch4, 'CO': co, 'CO2': co2, 'Clouds': clouds,  \
                                    'Observations': obs, 'Planet': planet,'TiO': tio, 'VO': vo, 'NH3': nh3,
                                     'Na': na, 'K': k, 'Hminus': hminus, 'FeH': feh, 'HCN': hcn})

display(ui, out)

Output()